In [36]:
import torch
import glob
import os
import pathlib
import cv2
import shutil

import numpy                as np
import pandas               as pd
import matplotlib.pyplot    as plt
import cv2                  as cv 
import matplotlib.pyplot    as plt
import torch.nn.functional  as F
import seaborn              as sns
import plotly.express       as px
import plotly.graph_objects as go

from random                  import sample
from plotly.subplots         import make_subplots
from google.colab            import drive
from google.colab.patches    import cv2_imshow
from skimage                 import io
from PIL                     import Image
from torch                   import nn, optim
from torchvision             import datasets, transforms
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print("Versão:",torch.__version__)

Versão: 1.10.0+cu111


In [37]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [38]:
dir_path_train = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/DatasetBrainTumor_70_30/Train_70'
dir_path_test = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/DatasetBrainTumor_70_30/Test_30'

In [39]:
transform_file = transforms.Compose(
    [
     transforms.Resize([128,128]),
     transforms.ToTensor()
    ]
)

In [40]:
train_dataset = datasets.ImageFolder(dir_path_train, transform=transform_file)
test_dataset = datasets.ImageFolder(dir_path_test, transform=transform_file)

### Pre-processamento de dados

In [41]:
def get_class_distribution(dataset_obj):
  count_dict = {k:0 for k, v in dataset_obj.class_to_idx.items()}
  for i in dataset_obj:
    y_labels = i[1]
    y_labels = idx2class[y_labels]
    count_dict[y_labels] += 1
  return count_dict

In [42]:
idx2class = {v: k for k, v in train_dataset.class_to_idx.items()}
df_treino = pd.DataFrame.from_dict([get_class_distribution(train_dataset)]).melt()

idx2class = {v: k for k, v in test_dataset.class_to_idx.items()}
df_teste = pd.DataFrame.from_dict([get_class_distribution(test_dataset)]).melt()

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(
    labels=df_treino['variable'],
     values=df_treino['value'],
      pull=[0, 0, 0.1, 0]),
       1, 1)

fig.add_trace(go.Pie(
    labels=df_teste['variable'],
     values=df_teste['value'],
      pull=[0, 0, 0.1, 0]),
       1, 2)

fig.update_traces(hole=.4,
    hoverinfo='label+percent',
                  textinfo='value',
                  textfont_size=12.5,
                  marker=dict(
                      line=dict(
                          color='#000000',
                           width=1)
                      )
                  )

fig.update_layout(
    title_text = 'Proporção da base de dados de treino e teste',
    annotations=[dict(text='Treino', x=0.181, y=0.5, font_size=20, showarrow=False),
                 dict(text='Teste', x=0.80, y=0.5, font_size=20, showarrow=False)]
)

fig.show()

## Undersample

In [44]:
path_glioma = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/DatasetBrainTumor_70_30/Train_70/glioma_tumor/'
print("Existe?....{}".format(os.path.exists(path_glioma)))
files = os.listdir(path_glioma)

quantidade_de_arquivos = os.listdir(path_glioma)
total = len(quantidade_de_arquivos)
diferenca = total - total_nao_tumores

for file in sample(files, diferenca):
  shutil.move(path_glioma + file, '/content/gdrive/MyDrive/MRI_TUMOR_CNN/DatasetBrainTumor_70_30/Train_Copy/Glioma_copy')

Existe?....True


In [45]:
path_meningioma = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/DatasetBrainTumor_70_30/Train_70/meningioma_tumor/'
print("Existe?....{}".format(os.path.exists(path_meningioma)))
files = os.listdir(path_meningioma)

quantidade_de_arquivos = os.listdir(path_meningioma)
total = len(quantidade_de_arquivos)
diferenca = total - total_nao_tumores

for file in sample(files, diferenca):
  shutil.move(path_meningioma + file, '/content/gdrive/MyDrive/MRI_TUMOR_CNN/DatasetBrainTumor_70_30/Train_Copy/Meningioma_copy')

Existe?....True


In [47]:
path_pituitary = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/DatasetBrainTumor_70_30/Train_70/pituitary_tumor/'
print("Existe?....{}".format(os.path.exists(path_pituitary)))
files = os.listdir(path_pituitary)

quantidade_de_arquivos = os.listdir(path_pituitary)
total = len(quantidade_de_arquivos)
diferenca = total - total_nao_tumores

for file in sample(files, diferenca):
  shutil.move(path_pituitary + file, '/content/gdrive/MyDrive/MRI_TUMOR_CNN/DatasetBrainTumor_70_30/Train_Copy/Pituitary_copy')

Existe?....True


In [48]:
dir_path_train = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/DatasetBrainTumor_70_30/Train_70'
dir_path_test = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/DatasetBrainTumor_70_30/Test_30'

In [49]:
train_dataset = datasets.ImageFolder(dir_path_train, transform=transform_file)
test_dataset = datasets.ImageFolder(dir_path_test, transform=transform_file)

In [50]:
idx2class = {v: k for k, v in train_dataset.class_to_idx.items()}
df_treino = pd.DataFrame.from_dict([get_class_distribution(train_dataset)]).melt()

idx2class = {v: k for k, v in test_dataset.class_to_idx.items()}
df_teste = pd.DataFrame.from_dict([get_class_distribution(test_dataset)]).melt()

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(
    labels=df_treino['variable'],
     values=df_treino['value'],
      pull=[0, 0, 0.1, 0]),
       1, 1)

fig.add_trace(go.Pie(
    labels=df_teste['variable'],
     values=df_teste['value'],
      pull=[0, 0, 0.1, 0]),
       1, 2)

fig.update_traces(hole=.4,
    hoverinfo='label+percent',
                  textinfo='value',
                  textfont_size=12.5,
                  marker=dict(
                      line=dict(
                          color='#000000',
                           width=1)
                      )
                  )

fig.update_layout(
    title_text = 'Proporção da base de dados de treino e teste',
    annotations=[dict(text='Treino', x=0.181, y=0.5, font_size=20, showarrow=False),
                 dict(text='Teste', x=0.80, y=0.5, font_size=20, showarrow=False)]
)

fig.show()